In [16]:
import pandas as pd

# Archivo separado por comas
df_final = pd.read_csv('df_final_demo.txt', sep=',')
df_experiments = pd.read_csv('df_final_experiment_clients.txt', sep=',')
df_webdata_1 = pd.read_csv('df_final_web_data_pt_1.txt', sep=',')
df_webdata_2 = pd.read_csv('df_final_web_data_pt_2.txt', sep=',')

In [17]:
### df_experiments ###

In [18]:
# Cambiamos los datos de Variation NA por 'Not participant'

df_experiments['Variation'] = df_experiments['Variation'].fillna('Not participant')
df_experiments['Variation'].value_counts()

Variation
Test               26968
Control            23532
Not participant    20109
Name: count, dtype: int64

In [19]:
### df_webdatas ###

In [20]:
# Primero unimos los df_webdata_1 y _2 en un único dataframe y eliminamos duplicados:

df_webdata = pd.concat([df_webdata_1, df_webdata_2], ignore_index=True).drop_duplicates()

df_webdata.head(11)

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43


In [21]:
# Contar el número de visitor_id únicos por cada visit_id
unique_visitor_count = df_webdata.groupby('visit_id')['visitor_id'].nunique()

# Verificar si todos los visit_id tienen un único visitor_id
unique_relationship = unique_visitor_count.max() == 1

print(f"La relación visit_id-visitor_id es única: {unique_relationship}")

# Como la relación es única, podemos eliminar la columna visitor_id

df_webdata = df_webdata.drop(columns=['visitor_id'])

La relación visit_id-visitor_id es única: True


In [22]:
# Agrupamos los datos del df_webdata para que sea más fácil de analizar:

# Aseguramos que 'date_time' es de tipo datetime
df_webdata['date_time'] = pd.to_datetime(df_webdata['date_time'])

# Agrupa por client_id y visit_id para calcular la duración de cada visita
df_duration = df_webdata.groupby(['client_id', 'visit_id']).agg(
    duration=('date_time', lambda x: x.max() - x.min())
).reset_index()

# Convertimos la duración a formato hh:mm:ss
df_duration['duration'] = df_duration['duration'].apply(
    lambda x: str(pd.to_timedelta(x)).split()[-1]
)

print(df_duration.head())

   client_id                      visit_id  duration
0        169  749567106_99161211863_557568  00:03:33
1        336  649044751_80905125055_554468  00:15:48
2        546    731811517_9330176838_94847  00:02:13
3        555  637149525_38041617439_716659  00:02:38
4        647   40369564_40101682850_311847  00:06:17


In [23]:
# Ahora, crearemos un DataFrame que cuente las ocurrencias de cada valor en process_step para cada combinación de client_id y visit_id

df_steps_count = df_webdata.pivot_table(
    index=['client_id', 'visit_id'],
    columns='process_step',
    aggfunc='size',
    fill_value=0
).reset_index()


# Obtener lista de columnas actuales
columns = list(df_steps_count.columns)

# Mover 'confirm' al final
columns.append(columns.pop(columns.index('confirm')))

# Reordenar el DataFrame con las columnas en el nuevo orden
df_steps_count = df_steps_count[columns]
df_steps_count.head()

process_step,client_id,visit_id,start,step_1,step_2,step_3,confirm
0,169,749567106_99161211863_557568,1,1,1,1,1
1,336,649044751_80905125055_554468,2,0,0,0,0
2,546,731811517_9330176838_94847,1,1,1,1,1
3,555,637149525_38041617439_716659,1,1,1,1,1
4,647,40369564_40101682850_311847,1,1,1,1,1


In [24]:
# Combinamos la duración con el conteo de steps
df_webdata_grouped = pd.merge(df_duration, df_steps_count, on=['client_id', 'visit_id'])

df_webdata_grouped.head()

# Unir df_webdata con df_experiment utilizando 'client_id' como clave

df_webdata_grouped = df_webdata_grouped.merge(df_experiments[['client_id', 'Variation']], on='client_id', how='left')

df_webdata_grouped = df_webdata_grouped.rename(columns={'Variation': 'variation'})

In [25]:
# Filtrar el DataFrame por client_id 9988021 usando query
df_filtered = df_webdata_grouped.query('client_id == 9988021')

# Mostrar los resultados filtrados
df_filtered

,client_id,visit_id,duration,start,step_1,step_2,step_3,confirm,variation
158917,9988021,219729223_42518459208_211914,01:57:16,7,0,0,0,0,Test
158918,9988021,781255054_21935453173_531117,00:10:45,1,1,3,3,0,Test


In [26]:
### df_final

In [27]:
# Unir df_final con df_experiment utilizando 'client_id' como clave

df_final_clients = df_final.merge(df_experiments[['client_id', 'Variation']], on='client_id', how='left')

In [28]:
df_final_clients.head()

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0,Test
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0,Control
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0,Test
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0,Test
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0,Control


In [29]:
# cambiamos el nombre de la columna gender y limpiamos sus datos

df_final_clients = df_final_clients.rename(columns={'Variation': 'variation'})
df_final_clients = df_final_clients.rename(columns={'gendr': 'gender'})
df_final_clients['gender'] = df_final_clients['gender'].replace('X', 'U')

